In [1]:
using Plots

In [2]:
using Gen, Memoize, Random

In [3]:
using StatsBase

In [111]:
using Statistics

## In this notebook we will implement a  Theory Of Mind agents on a version of Poker

Kuhn poker is an extremely simplified form of poker developed by Harold W. Kuhn as a simple model zero-sum two-player imperfect-information game, amenable to a complete game-theoretic analysis. In Kuhn poker, the deck includes only three playing cards, for example a King, Queen, and Jack. One card is dealt to each player, which may place bets similarly to a standard poker. If both players bet or both players pass, the player with the higher card wins, otherwise, the betting player wins.

#### Rules

In conventional poker terms, a game of Kuhn poker proceeds as follows:

Each player antes 1.
Each player is dealt one of the three cards, and the third is put aside unseen.
Player one can check or bet 1.
* If player one checks then player two can check or bet 1.
    * If player two checks there is a showdown for the pot of 2 (i.e. the higher card wins 1 from the other player).
    * If player two bets then player one can fold or call.
        * If player one folds then player two takes the pot of 3 (i.e. winning 1 from player 1).
        * If player one calls there is a showdown for the pot of 4 (i.e. the higher card wins 2 from the other player).
* If player one bets then player two can fold or call.
    * If player two folds then player one takes the pot of 3 (i.e. winning 1 from player 2).
    * If player two calls there is a showdown for the pot of 4 (i.e. the higher card wins 2 from the other player).

#### Optimal strategy


The game has a mixed-strategy Nash equilibrium; when both players play equilibrium strategies, the first player should expect to lose at a rate of −1/18 per hand (as the game is zero-sum, the second player should expect to win at a rate of +1/18). There is no pure-strategy equilibrium.

Kuhn demonstrated there are infinitely many equilibrium strategies for the first player, forming a continuum governed by a single parameter. In one possible formulation, player one freely chooses the probability $\alpha$ $\in$ [0,1/3] with which he will bet when having a Jack (otherwise he checks; if the other player bets, he should always fold). When having a King, he should bet with the probability of $3\alpha$ (otherwise he checks; if the other player bets, he should always call). He should always check when having a Queen, and if the other player bets after this check, he should call with the probability of $\alpha$ +1/3.

3The second player has a single equilibrium strategy: Always betting or calling when having a King; when having a Queen, checking if possible, otherwise calling with the probability of 1/3; when having a Jack, never calling and betting with the probability of 1/3.

### Constants

In [4]:
INVALID_VALUE = -100

-100

In [5]:
JACK = 1
QUEEN = 2
KING = 3
FULL_DECK = [JACK, QUEEN, KING]

3-element Array{Int64,1}:
 1
 2
 3

In [6]:
FOLD = -1
CHECK = 0
BET = 1
ACTIONS = [FOLD, CHECK, BET]

3-element Array{Int64,1}:
 -1
  0
  1

### Utility functions

In [7]:
function get_remaining_cards(card)
   return setdiff(FULL_DECK, [card])
end

get_remaining_cards (generic function with 1 method)

In [8]:
function reward(my_card::Int, buddy_card::Int)
    return my_card > buddy_card ? 1 : -1
end

reward (generic function with 1 method)

In [9]:
function reward(my_card::Int, buddy_card::Int, my_action::Int, buddy_action::Int)
    if my_action == FOLD
        return -1
    end
    if buddy_action == FOLD
        return 1
    end
    
    bet_factor = my_action == BET || buddy_action == BET ? 2 : 1
    
    return bet_factor * reward(my_card, buddy_card)
end

reward (generic function with 2 methods)

In [10]:
function expected_reward(my_card::Int, my_action::Int, buddy_action::Int, opp_cards_distribution=[0.5, 0.5])
    optional_buddy_cards = get_remaining_cards(my_card)
    rewards = map(optional_buddy_card -> reward(my_card, optional_buddy_card, my_action, buddy_action), optional_buddy_cards)
    expected_reward = rewards' * opp_cards_distribution
end

expected_reward (generic function with 2 methods)

### Some tests

In [11]:
expected_reward(JACK, CHECK, BET)

-2.0

In [12]:
expected_reward(JACK, FOLD, BET)

-1.0

In [13]:
expected_reward(QUEEN, FOLD, BET)

-1.0

In [14]:
expected_reward(QUEEN, CHECK, BET)

0.0

In [15]:
expected_reward(KING, FOLD, BET)

-1.0

In [16]:
expected_reward(KING, CHECK, BET)

2.0

#### We want to design an algorithm that infer the posterior for policy of check or bet, and the policy of fold or call

#### For each player we want to compute two vectors:
<ol>
    <li>Check or bet probability</li>
    <li>Fold or call probability</li>
</ol>

The rewards of fold or call is pretty much stright forward.<br>
Compute the expectation of reward given your card.<br>
Next stage can be based on computed cards distribution given opp_action is BET

In [17]:
@gen function fold_or_call(card, unused_flag)
    call = @trace(bernoulli(0.5), :call)    
    if call
        r = expected_reward(card, CHECK, BET)
    else
        r = expected_reward(card, FOLD, BET)
    end
    @trace(bernoulli(exp(r)), :r)
end;

The rewards of check or bet is less straight forward<br>
It's derived from the next optional buddy reaction to our action<br>
We can use opponent's fold_or_call model to derive the rewards in case we choose BET<br>
And use opponent's check_or_bet model to derive the rewards in case we choose CHECK and then our fold_or_call<br>

last_check argument controlד the amount of check_or_bet computation in a row (we want to cut it after 2 (rules of the game))

In [18]:
@gen function check_or_bet(card, last_check=false)
    bet = @trace(bernoulli(0.5), :bet)
    optional_buddy_cards = get_remaining_cards(card)
    if bet
        r = 0
        for opp_card = optional_buddy_cards
            
            # theory of mind about opponent's fold-cold policy
            calls = run_episodic(fold_or_call, opp_card, :call)
            call_ratio = sum(calls) / length(calls)
            
            # opp will call
            r_opp_will_call = reward(card, opp_card, BET, CHECK)

            # opp will fold
            r_opp_will_fold = reward(card, opp_card, BET, FOLD)
           
            r_opp_card = call_ratio * r_opp_will_call + (1-call_ratio)*r_opp_will_fold
            
            # expectation over cards, later we may can use learned probabilities
            r += 0.5 * r_opp_card
        end
    else
        r = 0
        for opp_card = optional_buddy_cards
            if last_check # cut the computation cycle
                r_opp_card = reward(card, opp_card, CHECK, CHECK)
            else # the opponent can check again or bet
                # theory of mind about opponent's check-bet policy
                bets = run_episodic(check_or_bet, opp_card, :bet, true)
                bet_ratio = sum(bets) / length(bets)

                
                # check if i will call or fold
                
                # theory of mind about mine fold-cold policy
                calls = run_episodic(fold_or_call, card, :call)
                call_ratio = sum(calls) / length(calls)
                
                # i will call
                r_opp_will_bet_i_will_call = reward(card, opp_card, CHECK, BET)
                
                # i will fold
                r_opp_will_bet_i_will_fold = reward(card, opp_card, FOLD, BET)
                
                r_opp_will_bet = call_ratio * r_opp_will_bet_i_will_call + (1-call_ratio)*r_opp_will_bet_i_will_fold
                # opp will bet
                r_opp_will_bet = reward(card, opp_card, CHECK, CHECK)
                
                # opp will check
                r_opp_will_check = reward(card, opp_card, CHECK, CHECK)

                r_opp_card = bet_ratio * r_opp_will_bet + (1-bet_ratio)*r_opp_will_check
            end
            r += 0.5 * r_opp_card
        end
        
    end
    @trace(bernoulli(exp(r)), :r)
end;

### Generic Gen infernce methods made by David in bob-alice-musings

In [19]:
@memoize function run_episodic(model, card, sym, last_check=false, niter=1000)
    observations = Gen.choicemap()
    observations[:r] = true

    trace, _ = Gen.generate(model, (card, last_check), observations)
    values = []
    for i = 1:niter
        trace, _ = Gen.mh(trace, select(sym))
        push!(values, get_choices(trace)[sym])
    end
    return values
end
empty!(memoize_cache(run_episodic));

In [20]:
function genepisodic(model, card, sym, last_check = false, niter=5000)
    nburn = niter%10
    values = run_episodic(model, card, sym, last_check, nburn + niter)[nburn+1:end]
    return sum(values)/length(values)
end;

In [21]:
function simulate(model, card, sym, last_check = false, niter=5000)
    nburn = niter%10
    values = run_episodic(model, card, sym, last_check, nburn + niter)[nburn+1:end]
    return sample(values)
end

simulate (generic function with 3 methods)

### Some tests

### for fold_or_call

In [22]:
genepisodic(fold_or_call, JACK, :call)

0.263

In [23]:
sum([simulate(fold_or_call, JACK, :call) for _ in 1:100]) / 100

0.21

In [24]:
genepisodic(fold_or_call, QUEEN, :call)

0.7364

In [25]:
sum([simulate(fold_or_call, QUEEN, :call) for _ in 1:100]) / 100

0.75

In [26]:
genepisodic(fold_or_call, KING, :call)

0.9438

In [27]:
sum([simulate(fold_or_call, KING, :call) for _ in 1:100]) / 100

0.93

### for check_or_bet

In [28]:
genepisodic(check_or_bet, JACK, :bet)

0.3754

In [29]:
genepisodic(check_or_bet, QUEEN, :bet)

0.4378

In [30]:
genepisodic(check_or_bet, KING, :bet)

0.6276

### for check_or_bet - last_check

In [31]:
genepisodic(check_or_bet, JACK, :bet, true)

0.3736

In [32]:
genepisodic(check_or_bet, QUEEN, :bet, true)

0.4356

In [33]:
genepisodic(check_or_bet, KING, :bet, true)

0.6098

### Simulators that run the inference

In [34]:
function compute_policy_fold_or_call(card, last_check)
    p_call = genepisodic(fold_or_call, card, :call, last_check)
    return [1-p_call, p_call]
end

compute_policy_fold_or_call (generic function with 1 method)

In [35]:
function compute_policy_check_or_bet(card, last_check)
    p_bet = genepisodic(check_or_bet, card, :bet, last_check)
    return [1-p_bet, p_bet]
end

compute_policy_check_or_bet (generic function with 1 method)

#### Activates the proper compute_policy

In [173]:
function compute_policy(card, previous_player_betted=false)
    if previous_player_betted
        return compute_policy_fold_or_call(card, !previous_player_betted)
    else
        return compute_policy_check_or_bet(card, !previous_player_betted) 
    end
end

compute_policy (generic function with 3 methods)

In [37]:
function sample_check_bet(policy)
   sample([CHECK,BET], Weights(policy)) 
end

sample_check_bet (generic function with 1 method)

In [38]:
function sample_fold_call(policy)
   sample([FOLD,CHECK], Weights(policy)) 
end

sample_fold_call (generic function with 1 method)

#### Activates the proper sample_action

In [39]:
function sample_action(policy, previous_player_betted=false)
    if previous_player_betted
        return sample_fold_call(policy)
    else
        return sample_check_bet(policy) 
    end
end

sample_action (generic function with 2 methods)

### This method simulates the logic of single round, lets both player to simulate an action and returns the score of the round (positive for first player winning)

In [174]:
function poker_round(first_player_card, second_player_card, history)
    score = 0
    
    first_player_policy = compute_policy(first_player_card)
    first_player_move = sample_action(first_player_policy)
    
    second_player_policy  = compute_policy(second_player_card, first_player_move == BET)
    second_player_move = sample_action(second_player_policy, first_player_move == BET)
    
    round_history = [((first_player_card, first_player_move, false), (second_player_card, second_player_move, first_player_move == BET))]
    
    doubled_pot = first_player_move == BET || second_player_move == BET
    if second_player_move == FOLD
        round_record = (round_history, 1)
        push!(history, round_record)
        return 1
    end
    if second_player_move == BET
        doubled_pot = true
        first_player_policy = compute_policy(first_player_card, second_player_move == BET)
        first_player_move = sample_action(first_player_policy, second_player_move == BET)
        
        push!(round_history, ((first_player_card, first_player_move, false),(INVALID_VALUE, INVALID_VALUE, false)))
        if first_player_move == FOLD
            return -1
        end
    end
    if first_player_card > second_player_card
        score =  1 * (1 + doubled_pot)
    else
        score = -1 * (1 + doubled_pot)
    end
    round_record = (round_history, score)
    push!(history, round_record)
    return score
end

poker_round (generic function with 1 method)

#### This method simulates simultaneously -num_of_expirements- a whole game that contains -num_of_rounds-
#### Returns a list of accumalated average score over all rounds and over all games

In [68]:
function simulate_multiple_games(num_of_rounds = 10, num_of_expirements = 10)
    total_scores = [0 for _ in 1:num_of_expirements]
    history = []
    avg_scores_acc = []
    for i in 1:num_of_rounds
        card_pairs = [sample(FULL_DECK, 2; replace=false) for _ in 1:num_of_expirements]
        scores = [poker_round(first_player_card, second_player_card, history) for (first_player_card, second_player_card) in card_pairs]
        total_scores += scores
        avg_scores = total_scores / i
        push!(avg_scores_acc, avg_scores)
    end    
    return avg_scores_acc
end

game (generic function with 3 methods)

#### This method plots the accumalated average score , we can see that all simulation converages to same value

In [145]:
function plot_game_results(num_of_rounds, num_of_expirements)
    results = transpose(reduce(hcat, simulate_multiple_games(num_of_rounds, num_of_expirements)))
    plot(1:num_of_rounds, results, label=nothing, color="blue", alpha=(5/num_of_expirements))
    average_total_score , std_total_score = round(mean(results[num_of_rounds, :]), digits=3), round(std(results[num_of_rounds, :]), digits=3)
    hspan!([average_total_score-0.008, average_total_score+0.008], label="average total score: $average_total_score +- $std_total_score", color="red")
end

plot_game_results (generic function with 1 method)

In [ ]:
plot_game_results(1000,200)

In [ ]:
plot_game_results(5000, 200)

In [ ]:
plot_game_results(10000, 200)

### We can see that when two probablistic players play one against each other the eq is approximatlly -0.22

## Lets see the results one Probablistic player plays agains nash-eq strategy

### The implementation of the Nash-equilibrium strategies are well-define (defined in the start of the notebook)

#### This method is implementation of first player Nash-equilibrium strategy

In [207]:
function first_player_nash_eq_strategy(card, opp_betting=false, deception_alpha=0.2)
   if card == JACK
        if opp_betting
            return FOLD    
        elseif rand() < deception_alpha
            return BET # bluff
        else
            return CHECK
        end
    elseif card == KING
        need_to_bet = rand() < 3 * deception_alpha
        if need_to_bet && !opp_betting
            return BET
        else   # slow playing
            return CHECK
        end
    else # queen
        need_to_call = rand() < deception_alpha + 1/3
        if !need_to_call && opp_betting
            return FOLD
        else
            return CHECK
        end
    end
end

first_player_nash_eq_strategy (generic function with 3 methods)

#### This method is implementation of second player Nash-equilibrium strategy

In [161]:
function second_player_nash_eq_strategy(card, opp_betting=false)
   if card == JACK
        if opp_betting
            return FOLD    
        elseif rand() < 1/3
            return BET # bluff
        else
            return CHECK
        end
    elseif card == KING
        if  opp_betting
            return CHECK
        else   # slow playing
            return BET
        end
    else # queen
        need_to_call = rand() < 1/3
        if !need_to_call && opp_betting
            return FOLD
        else
            return CHECK
        end
    end
end

second_player_nash_eq_strategy (generic function with 2 methods)

In [176]:
function probablistic_player_strategy(card, opp_betting=false)
    probablistic_player_policy  = compute_policy(card,opp_betting)
    probablistic_player_move = sample_action(probablistic_player_policy, opp_betting)
    return probablistic_player_move
end

probablistic_player_strategy (generic function with 2 methods)

In [195]:
function poker_round_generic(first_player_card, second_player_card, history, first_player_move_selection_strategy, second_player_move_selection_strategy)
    score = 0
    
    first_player_move = first_player_move_selection_strategy(first_player_card)
    second_player_move = second_player_move_selection_strategy(second_player_card, first_player_move == BET)
    
    round_history = [((first_player_card, first_player_move, false), (second_player_card, second_player_move, first_player_move == BET))]
    
    doubled_pot = first_player_move == BET || second_player_move == BET
    if second_player_move == FOLD
        round_record = (round_history, 1)
        push!(history, round_record)
        return 1
    end
    if second_player_move == BET
        doubled_pot = true
        first_player_move = first_player_move_selection_strategy(first_player_card, second_player_move == BET)
        
        push!(round_history, ((first_player_card, first_player_move, false),(INVALID_VALUE, INVALID_VALUE, false)))
        if first_player_move == FOLD
            return -1
        end
    end
    if first_player_card > second_player_card
        score =  1 * (1 + doubled_pot)
    else
        score = -1 * (1 + doubled_pot)
    end
    round_record = (round_history, score)
    push!(history, round_record)
    return score
end

poker_round_generic (generic function with 1 method)

In [193]:
function simulate_multiple_games_generic(num_of_rounds, num_of_expirements, first_player_move_selection_strategy=probablistic_player_strategy, second_player_move_selection_strategy=probablistic_player_strategy)
    total_scores = [0 for _ in 1:num_of_expirements]
    history = []
    avg_scores_acc = []
    for i in 1:num_of_rounds
        card_pairs = [sample(FULL_DECK, 2; replace=false) for _ in 1:num_of_expirements]
        scores = [poker_round_generic(first_player_card, second_player_card, history, first_player_move_selection_strategy, second_player_move_selection_strategy) for (first_player_card, second_player_card) in card_pairs]
        total_scores += scores
        avg_scores = total_scores / i
        push!(avg_scores_acc, avg_scores)
    end    
    return avg_scores_acc
end

simulate_multiple_games_generic (generic function with 3 methods)

In [182]:
function plot_game_results_generic(num_of_rounds, num_of_expirements, first_player_move_selection_strategy=probablistic_player_strategy, second_player_move_selection_strategy=probablistic_player_strategy)
    results = transpose(reduce(hcat, simulate_multiple_games_generic(num_of_rounds, num_of_expirements, first_player_move_selection_strategy, second_player_move_selection_strategy)))
    plot(1:num_of_rounds, results, label=nothing, color="blue", alpha=(5/num_of_expirements))
    average_total_score , std_total_score = round(mean(results[num_of_rounds, :]), digits=3), round(std(results[num_of_rounds, :]), digits=3)
    hspan!([average_total_score-0.008, average_total_score+0.008], label="average total score: $average_total_score +- $std_total_score", color="red")
end

plot_game_results_generic (generic function with 3 methods)

In [ ]:
plot_game_results_generic(1000,200, probablistic_player_strategy, probablistic_player_strategy)

In [ ]:
plot_game_results_generic(1000,200, first_player_nash_eq_strategy, probablistic_player_strategy)

In [ ]:
plot_game_results_generic(1000,200, probablistic_player_strategy, second_player_nash_eq_strategy)

In [ ]:
plot_game_results_generic(1000,200, first_player_nash_eq_strategy, second_player_nash_eq_strategy)

In [ ]:
plot_game_results_generic(5000,200, probablistic_player_strategy, probablistic_player_strategy)

In [ ]:
plot_game_results_generic(5000,200, first_player_nash_eq_strategy, probablistic_player_strategy)

In [ ]:
plot_game_results_generic(5000,200, probablistic_player_strategy, second_player_nash_eq_strategy)

In [ ]:
plot_game_results_generic(5000,200, first_player_nash_eq_strategy, second_player_nash_eq_strategy)